In [1]:
# Import Statements
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String
import EnvMap
from numba import jit

In [2]:
class compare():
    def __init__(self,pre_map,timestep):
        self.pre_map = pre_map        
        self.timestep = timestep
    def compare(self,new_map):
        new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
        new_map[(self.pre_map == [70]) & (new_map == [100])] = [70]
        return new_map

In [3]:
# Initialization Stuff

rospy.init_node('liveproc')

listener = tf.TransformListener()
pub = rospy.Publisher('updated_map', Map, queue_size=10)
pre_timestep = None

occ_map = None
up_occ_map = None
map_resolution = 0.5
origin = None
robot_x = 0
robot_y = 0
map_width = 0

map_height = 0
camera_range = 85/map_resolution
overlay = True
display_robot_yaw = False
display_rect = True
display_map = False
display_info = False
tri_width = 5
dist_error = 0.15

np.set_printoptions(threshold=sys.maxsize)

In [4]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [5]:
def disp_map(map_img):
    map_img = np.swapaxes(map_img, 0, 1)
    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img)
    cv.waitKey(1)

In [6]:
def transform(img):
    
    global bearings
    global areas
    global tags    
    
    bridge = cv_bridge.CvBridge()
    
    areas = []
    bearings = []
    tags = []
    
    width = img.height
    height = img.width
    
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  
    
    
    image = pp(img,width,height)
    
    img2= dbscan(image, img)
    
    return img2

In [7]:
def pp(image, width, height):
    
    image = np.array(image[0:width][0:int(height*0.65)])
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)

    return(image)

In [8]:
def dbscan(image, original):
    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))

    if len(X) > 0:
        start_time = time.time()
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while len(sort) > 0 and sort[x][2] == -1:
            del sort[x]

        if len(sort) > 0:
            unique_labels = sorted(set(labels))
            if unique_labels[0] == -1:
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

            i = 0
            x = 0
            l = []
            tb = None
            while x < len(unique_labels):
                if i < len(sort) and sort[i][2] == x:
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color_hsv(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original)
                    l = []
            original = tb
    return original

In [9]:
def color_hsv(mat, image):
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sum_hue = 0
    sum_sat = 0
    sum_val = 0
    mat_add = 0
    for x in range(len(mat)):
        if image [mat[x][1]] [mat[x][0]] [2] > 5:
            mat_add = mat_add+1
            sum_val += image [mat[x][1]] [mat[x][0]] [2]
            sum_sat += image [mat[x][1]] [mat[x][0]] [1]
            sum_hue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sum_sat/mat_add
        val = sum_val/mat_add
        hue = sum_hue/mat_add
        #return (hue, sat, val)
        if sat > 5 and val > 5 and (hue < 60 or hue >= 170):
            return 'red'
        if sat > 5 and val > 5 and (70 < hue < 90):
            return 'green'
        else:
            return hue, sat, val
    return None

In [10]:
def rectangle(mat,color,image,original):
    
    global bearings
    global overlay
    global occ_map
    global tags
    global display_info
    global map_resolution
    global areas
    
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if width < height < 4*width and width*height > 45:
            if color =='green':
                tag = 2
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
                tags.append(tag)
            elif color =='red':
                tag = 3
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            cv.putText(original,'bearing: '+ str(round(bearing,2)),(np.int0(x+width/2),np.int0(y+height/2.5)), font, 0.5,(0,0,0),2,cv.LINE_AA)
            bearings.append(bearing)
            area = min_rect[1][0]*min_rect[1][1]
            areas.append(area)
#             print("Area: " + str(area) + " and Tag: " + str(tag) +
#                   "\nEstimated Distance: " + str(2*map_resolution*456.7946397925*math.pow(area,-0.4705300238)))
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        if tag == 0:
            image[y:y+height][x:x+width]=[0,0,0]
        return original
    else:
        if width < height < 4*width and width*height > 45:
            if color =='green':
                tag = 2
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            elif color =='red':
                tag = 3
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
                tags.append(tag)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (min_rect[0][0]-640.0)/1280.0*80.0
            bearings.append(bearing)
            area = min_rect[1][0]*min_rect[1][1]
            areas.append(area)
            
            if display_info and occ_map.header.seq % 5 == 0:
                print bearings
                print tags
        return image

In [11]:
def draw_background_objects(map_img, map_np):
    
    global origin
    global robot_x
    global robot_y
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
# Color Assignment Section
    map_img[map_np == -1] = [0]    #Change to [255,255,255] for color to [0] for grayscale
    map_img[map_np == 0] = [0]     #Change to [255,255,255] for color to [0] for grayscale
    map_img[map_np == 100] = [100]         #Change to [0,0,0] for color to [100] for grayscale
    origin_color =  [10]                #Change to [0,0,255] for color to [0] for grayscale
    robot_color = [10]                  #Change to [0,0,255] for color to [0] for grayscale
# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color
# Robot (Drawn as a square)
    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color
    
    return map_img

In [12]:
def draw_ray(map_img,slices):
    
    global display_robot_yaw
    global bearings
    global tags
    global areas
    global robot_x
    global robot_y
    global map_resolution
    global areas
    
    slices_size = len(slices)
    
    if not display_robot_yaw:
        for i in range(int(slices_size)):
            try:
                arr = slices[i]
                appr_dist_low = (1.0-2.5*dist_error)*map_resolution*2*456.7946397925*math.pow(areas[i],-0.4705300238)
                for ray in arr:
                    if ray[3] > appr_dist_low: 
                        if ray[2] == -1 or ray[2] == 0:
                            if False: #display true if you want to see slice
                                map_img[ray[1]][ray[0]] = [255,0,0]#Change to [255,0,0] for color to [0] for grayscale
                        if ray[2] == 100:
                            if tags[i] == 2:
                                map_img[ray[1]][ray[0]]= [30] #Change to [0,255,0] for color to 0,[30 for grayscale]
                            if tags[i] == 3: 
                                map_img[ray[1]][ray[0]]= [70] #Change to [0,0,255] for color to 0,[50 for grayscale]
            except Exception as ex:
                print ex
    else:
        for arr in slices[-1]:
            map_img[arr[1]][arr[0]] = [0,255,0]
    
    return map_img

In [13]:
def get_mapnbot_info(trans,robot_yaw):

    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    global map_np
    
    map_array = occ_map.data
    
    map_width = occ_map.info.width
    map_height = occ_map.info.height
    
    map_np = np.reshape(np.array(map_array),(map_height,map_width))

    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
    
    return map_np, robot_yaw

In [14]:
def insidetriangle((x1,x2,x3),(y1,y2,y3),map_np):

    global robot_x
    global robot_y
    
    xs=np.array((x1,x2,x3),dtype=float)
    ys=np.array((y1,y2,y3),dtype=float)

    # The possible range of coordinates that can be returned
    x_range=np.arange(np.min(xs),np.max(xs)+1)
    y_range=np.arange(np.min(ys),np.max(ys)+1)

    # Set the grid of coordinates on which the triangle lies. The centre of the
    # triangle serves as a criterion for what is inside or outside the triangle.
    X,Y=np.meshgrid( x_range,y_range )
    xc=np.mean(xs)
    yc=np.mean(ys)

    # From the array 'triangle', points that lie outside the triangle will be
    # set to 'False'.
    triangle = np.ones(X.shape,dtype=bool)
    for i in range(3):
        ii=(i+1)%3
        if xs[i]==xs[ii]:
            include = X *(xc-xs[i])/abs(xc-xs[i]) > xs[i] *(xc-xs[i])/abs(xc-xs[i])
        else:
            poly=np.poly1d([(ys[ii]-ys[i])/(xs[ii]-xs[i]),ys[i]-xs[i]*(ys[ii]-ys[i])/(xs[ii]-xs[i])])
            include = Y *(yc-poly(xc))/abs(yc-poly(xc)) > poly(X) *(yc-poly(xc))/abs(yc-poly(xc))
        triangle*=include

    # Output: 2 arrays with the x- and y- coordinates of the points inside the
    # triangle.
    

    x_points = X[triangle]
    y_points = Y[triangle]

    points = np.column_stack((x_points.astype(int), y_points.astype(int)))

    itbuffer = np.empty(shape=((points.size/2),4), dtype=np.int32)
    itbuffer.fill(np.nan)

    itbuffer[:,0] = x_points
    itbuffer[:,1] = y_points
    itbuffer[:,2] = map_np[points[:,1],points[:,0]]

# Get distance values    

    deltax = itbuffer[:,0]-robot_x
    deltay = itbuffer[:,1]-robot_y
    
    deltax = map_resolution*deltax
    deltay = map_resolution*deltay
    
    
    deltax = np.square(deltax)
    deltay = np.square(deltay)

    
    dist_sq = deltax+deltay
    
    dist_sq = np.sqrt(dist_sq)
#     dist_sq = map_resolution*dist_sq
    
    
    itbuffer[:,3] = dist_sq
#     print robot_x,robot_y
#     print np.column_stack((deltax.astype(int),deltay.astype(int)))
    return itbuffer

In [15]:
def get_points(buoy_yaw, index):
    
    global robot_x
    global robot_y
    global camera_range
    global tri_width
    global areas
    global dist_error
    
    appr_dist_up = (1.0+2.0*dist_error)*2*456.7946397925*math.pow(areas[index],-0.4705300238)
    point_deltax = appr_dist_up * math.sin(math.radians(buoy_yaw)) # appr_dist_up
    point_deltay = appr_dist_up * math.cos(math.radians(buoy_yaw)) #appr_dist_up
    per_slope = -1.0/(point_deltay/point_deltax)
    point1 = np.array([round(robot_x), round(robot_y)], dtype=int)
    point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)], dtype=int)

    
    
    point3_x = (robot_x+point_deltax) + math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))
    point4_x = (robot_x+point_deltax) - math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))

    point3_y = robot_y+point_deltay + per_slope*(point3_x-(robot_x+point_deltax))
    point4_y = robot_y+point_deltay + per_slope*(point4_x-(robot_x+point_deltax))

    point3 = np.array([round(point3_x),round(point3_y)], dtype=int)
    point4 = np.array([round(point4_x),round(point4_y)], dtype=int)
    
    
    return point1,point2,point3,point4

In [16]:
def map_filter(trans, robot_yaw):
    
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    
    slices = []
    i = 0
    while i < len(bearings):
        
        
        buoy_yaw = robot_yaw + bearings[i]
        if buoy_yaw < 0:
            buoy_yaw = buoy_yaw + 360
#         print bearings
##############################################################################
# This is for visualization purposes.
# Set to false if you don't want to display the robot_yaw
# Delete it if it's irritating to look at
        if display_robot_yaw and occ_map.header.seq % 5 == 0:
            bearings.append(0.0)
            print "Robot_yaw: ", robot_yaw
            print "Buoy yaw: ", buoy_yaw
            print len(slices)
##############################################################################
        
        pt1, pt2, pt3, pt4 = get_points(buoy_yaw, i)    
        
#         sliced = createLineIterator(pt1.astype(int),pt2.astype(int),map_np)

        sliced = insidetriangle((pt3[0],pt1[0],pt4[0]),(pt3[1],pt1[1],pt4[1]),map_np)
        slices.append(np.array(sliced).astype(int))
#         print slices
        i+=1
    
    return slices

In [17]:
def get_map(slices):
    
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    global pre_timestep
    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))
    #Set to 1 when publishing, to 3 for visualization  
#     map_img_z = np.zeros((map_height, map_width,3))
    map_img_z = np.zeros((map_height, map_width,1))
    map_img_b = draw_background_objects(map_img_z,map_np)
    if pre_timestep != None:
        map_img_b = pre_timestep.compare(map_img_b)
    map_img_r = draw_ray(map_img_b,slices)
    
    
    
    return map_img_r

In [18]:
def callback(rect):

    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags
    global up_occ_map
    global pub
    global map_height
    global map_width
    global pre_timestep
    
    origin = occ_map.info.origin.position
    rect = transform(rect)
    if not rospy.is_shutdown():       
        now = rospy.Time.now()
        listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(1))
        (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
        euler = tf.transformations.euler_from_quaternion(rot)
        pixel_lines = map_filter(trans, math.degrees(euler[2]))
        map_image = get_map(pixel_lines)
        pre_timestep = compare(map_image, str(occ_map.header.seq))
#         map_obj = EnvMap.EnvMap(map_image,[int(origin.y),int(origin.x)])
#         buoy_data = map_obj.get_buoys()
        if True: #set to true if you want to publish the updated occupancy map
            up_occ_map.data = np.array(map_image).flatten()
            pub.publish(up_occ_map)
        if display_rect:
            disp_rect(rect)
        if display_map:
            disp_map(map_image)

In [19]:
def map_callback(p_map):
    
    global occ_map
    global up_occ_map
    
    occ_map = p_map    
    up_occ_map = p_map

In [ ]:
if __name__ == '__main__':
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()   

[ERROR] [1564604197.669277, 18787.668000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (439,427,1) 

[ERROR] [1564604197.780453, 18787.779000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi

[ERROR] [1564604199.600866, 18789.598000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (446,428,1) 

[ERROR] [1564604199.791770, 18789.788000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi

[ERROR] [1564604201.585333, 18791.580000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (448,428,1) 

[ERROR] [1564604201.702074, 18791.697000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi

[ERROR] [1564604203.673548, 18793.667000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (450,431,1) 

[ERROR] [1564604203.870770, 18793.863000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi

[ERROR] [1564604205.791461, 18795.765000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (451,431,1) 

[ERROR] [1564604205.923708, 18795.894000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 23, in callback
    pixel_lines = map_filt

...

[ERROR] [1564604682.850800, 19271.935000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (451,435,1) 

[ERROR] [1564604683.019283, 19272.104000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi

[ERROR] [1564604685.385229, 19274.467000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (451,435,1) 

[ERROR] [1564604685.553998, 19274.636000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi

[ERROR] [1564604688.058344, 19277.137000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (451,435,1) 

[ERROR] [1564604688.246454, 19277.326000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi

[ERROR] [1564604690.661377, 19279.738000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (451,435,1) 

[ERROR] [1564604690.849138, 19279.926000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi

[ERROR] [1564604693.400031, 19282.474000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pixel_lines)
  File "<ipython-input-17-af4c3a2b6907>", line 18, in get_map
    map_img_b = pre_timestep.compare(map_img_b)
  File "<ipython-input-2-6c442f3e4cb2>", line 6, in compare
    new_map[(self.pre_map == [30]) & (new_map == [100])] = [30]
ValueError: operands could not be broadcast together with shapes (407,423,1) (451,435,1) 

[ERROR] [1564604693.586036, 19282.660000]: bad callback: <function callback at 0x7f9220177848>
Traceback (most recent call last):
  File "/opt/ros/kinetic/lib/python2.7/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-18-6d1df56c51aa>", line 24, in callback
    map_image = get_map(pi